### Object Detection Using YOLO

In [33]:
import cv2

import numpy as np

# Load Yolo

net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")  

# dnn is deep neural network

classes = []   

# in classes you can specify objects that you want to detect 

# classes=['car','person','bicycle'] 

with open("coco.names", "r") as f:      

    classes = [line.strip() for line in f.readlines()]  
print(classes)
print(len(classes))

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']
80


In [35]:
import cv2
import numpy as np

# Load YOLO model
net = cv2.dnn.readNetFromDarknet("yolov3.cfg", "yolov3.weights")

# Get the names of all layers in the network
layer_names = net.getLayerNames()

# Get the output layer names
unconnected_out_layers = net.getUnconnectedOutLayers()

# Check if the output is a 2D array or a 1D array/scalar
if len(unconnected_out_layers.shape) == 2:
    output_layers = [layer_names[i[0] - 1] for i in unconnected_out_layers]
else:
    output_layers = [layer_names[i - 1] for i in unconnected_out_layers]

# Generate random colors for each class
colors = np.random.uniform(0, 255, size=(len(classes), 3))

print("Output layers:", output_layers)


Output layers: ['yolo_82', 'yolo_94', 'yolo_106']


In [86]:
img = cv2.imread("image1.jpg")   

# Enter your image name here that you want to detect

img = cv2.resize(img, None, fx=0.4, fy=0.4) #OPTIONAL

# Resizing the image. None refers to no fixed size. fx and fy are width and height .

height, width, channels = img.shape

In [88]:
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)  

# True refers to converting into rgb format since opencv uses bgr.

net.setInput(blob)  

# Passing blob image to yolo algo in network

outs = net.forward(output_layers)  

# Giving network to output layer for final result.

In [89]:
# Showing informations on the screen

class_ids = []

confidences = []

boxes = []

for out in outs:

    for detection in out:

        scores = detection[5:]

        class_id = np.argmax(scores)

        confidence = scores[class_id]

        if confidence > 0.5:

            # Object detected

            center_x = int(detection[0] * width)

            center_y = int(detection[1] * height)

            w = int(detection[2] * width)

            h = int(detection[3] * height)

            # Rectangle coordinates

            x = int(center_x - w / 2)

            y = int(center_y - h / 2)

            boxes.append([x, y, w, h])

            confidences.append(float(confidence))

            class_ids.append(class_id)

In [92]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4) 

# NMS - non max supression

#print(indexes)

font = cv2.FONT_HERSHEY_PLAIN

for i in range(len(boxes)):

    if i in indexes:

        x, y, w, h = boxes[i]

        label = str(classes[class_ids[i]])

        color = colors[i]

        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2) 

        # Draw rectangle around boxes. '2' is the width of box.

        cv2.putText(img, label, (x, y + 30), font, 3, color, 3)

        # Text in Box to label the object

In [98]:
cv2.imshow("Image", img) 

cv2.waitKey(0) 

 # waitkey stops the output

cv2.destroyAllWindows()